In [1]:
pip install mysql-connector-python pandas mlxtend matplotlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector
import pandas as pd

In [3]:
mydb=mysql.connector.connect(
    host="18.136.157.135",
    user="dm_team16",
    password="2o_hihiFeTRE",
    database="project_purchase_pattern_analysis"
)

In [4]:
pip install sqlalchemy pymysql

Note: you may need to restart the kernel to use updated packages.


In [5]:
query="select billno,itemname,quantity from mytable"
df=pd.read_sql(query,mydb)
df.head()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15204\709456807.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query,mydb)


,billno,itemname,quantity
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,WHITE METAL LANTERN,6
2,536365,CREAM CUPID HEARTS COAT HANGER,8
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6


In [6]:
print("total rows:",df.shape[0])
print("unique bills:",df['billno'].nunique())
print("unique items:",df['itemname'].nunique())
df.head(10)

total rows: 522064
unique bills: 21663
unique items: 4186


,billno,itemname,quantity
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6
1,536365,WHITE METAL LANTERN,6
2,536365,CREAM CUPID HEARTS COAT HANGER,8
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6
5,536365,SET 7 BABUSHKA NESTING BOXES,2
6,536365,GLASS STAR FROSTED T-LIGHT HOLDER,6
7,536366,HAND WARMER UNION JACK,6
8,536366,HAND WARMER RED POLKA DOT,6
9,536367,ASSORTED COLOUR BIRD ORNAMENT,32


In [7]:
print(df['quantity'].unique()[:10])

[ 6  8  2 32  3  4 24 12 48 18]


In [8]:
print(df.groupby(['billno','itemname'])['quantity'].sum().reset_index().head(10))

   billno                             itemname  quantity
0  536365       CREAM CUPID HEARTS COAT HANGER         8
1  536365    GLASS STAR FROSTED T-LIGHT HOLDER         6
2  536365  KNITTED UNION FLAG HOT WATER BOTTLE         6
3  536365       RED WOOLLY HOTTIE WHITE HEART.         6
4  536365         SET 7 BABUSHKA NESTING BOXES         2
5  536365   WHITE HANGING HEART T-LIGHT HOLDER         6
6  536365                  WHITE METAL LANTERN         6
7  536366            HAND WARMER RED POLKA DOT         6
8  536366               HAND WARMER UNION JACK         6
9  536367        ASSORTED COLOUR BIRD ORNAMENT        32


In [9]:
print("unique bills:",df['billno'].nunique())
print("inique items:",df['itemname'].nunique())

unique bills: 21663
inique items: 4186


In [10]:
df.rename(columns={'billno':'temp'},inplace=True)
df.rename(columns={'itemname':'billno','temp':'itemname'},inplace=True)

print(df[['billno','itemname','quantity',]].head())

                                billno itemname  quantity
0   WHITE HANGING HEART T-LIGHT HOLDER   536365         6
1                  WHITE METAL LANTERN   536365         6
2       CREAM CUPID HEARTS COAT HANGER   536365         8
3  KNITTED UNION FLAG HOT WATER BOTTLE   536365         6
4       RED WOOLLY HOTTIE WHITE HEART.   536365         6


In [11]:
import numpy as np

df ['billno']=df ['billno'].astype(str).str.strip()

df['itemname']=df['itemname'].astype(str).str.strip().str.upper()

df ['quantity']=pd.to_numeric(df['quantity'],errors='coerce')

basket=df.groupby(['billno', 'itemname'])['quantity'].sum().unstack().fillna(0)

basket=basket.applymap(lambda x: np.where(x>0,1,0))

print(basket.iloc[:5,:10])
print("total non zero values:", (basket !=0).sum().sum())

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15204\4149271386.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket=basket.applymap(lambda x: np.where(x>0,1,0))


itemname                    536365  536366  536367  536368  536369  536370  \
billno                                                                       
                                 0       0       0       0       0       0   
*Boombox Ipod Classic            0       0       0       0       0       0   
*USB Office Mirror Ball          0       0       0       0       0       0   
10 COLOUR SPACEBOY PEN           0       0       0       0       0       0   
12 COLOURED PARTY BALLOONS       0       0       0       0       0       0   

itemname                    536371  536372  536373  536374  
billno                                                      
                                 0       0       0       0  
*Boombox Ipod Classic            0       0       0       0  
*USB Office Mirror Ball          0       0       0       0  
10 COLOUR SPACEBOY PEN           0       0       0       0  
12 COLOURED PARTY BALLOONS       0       0       0       0  
total non zero values: 509

In [12]:
print((basket !=0).sum(axis=1).head(10))

billno
                                    592
*Boombox Ipod Classic                 1
*USB Office Mirror Ball               2
10 COLOUR SPACEBOY PEN              312
12 COLOURED PARTY BALLOONS          160
12 DAISY PEGS IN WOOD BOX            73
12 EGG HOUSE PAINTED WOOD            92
12 HANGING EGGS HAND PAINTED          7
12 IVORY ROSE PEG PLACE SETTINGS    146
12 MESSAGE CARDS WITH ENVELOPES     248
dtype: int64


In [13]:
pip install mlxtend

In [14]:
from mlxtend.frequent_patterns import apriori, association_rules

#find frequent itemset
frequent_items=apriori(basket,min_support=0.05,use_colnames=True)

#generate association rules
rules= association_rules(frequent_items,metric="lift", min_threshold=1)

print(frequent_items.head())
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

C:\Users\Lenovo\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


    support  itemsets
0  0.124701  (536544)
1  0.140707  (536592)
2  0.139752  (536876)
3  0.142140  (537237)
4  0.135452  (537240)
  antecedents consequents   support  confidence      lift
0    (536544)    (536592)  0.062112    0.498084  3.539866
1    (536592)    (536544)  0.062112    0.441426  3.539866
2    (536544)    (536876)  0.065456    0.524904  3.755981
3    (536876)    (536544)  0.065456    0.468376  3.755981
4    (536544)    (537237)  0.068801    0.551724  3.881542
5    (537237)    (536544)  0.068801    0.484034  3.881542
6    (536544)    (537240)  0.064023    0.513410  3.790360
7    (537240)    (536544)  0.064023    0.472663  3.790360
8    (536544)    (537434)  0.069756    0.559387  3.479337
9    (537434)    (536544)  0.069756    0.433878  3.479337


In [18]:
# FULL WEEK 3 PIPELINE (regenerate rules with lower support)

import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# 1) Load data
data = pd.read_csv("cleaned_purchase_data.csv")

# 2) Basket create
basket = data.groupby(['BillNo', 'Itemname'])['Quantity'].sum().unstack().fillna(0)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

# 3) Frequent itemsets - LOWER SUPPORT so more rules
frequent_items = apriori(
    basket,
    min_support=0.01,   # 1% support (previous 0.05 too high)
    use_colnames=True,
    max_len=2,
    low_memory=True
)

# 4) Association rules (NO filtering)
rules = association_rules(frequent_items, metric="lift", min_threshold=1)

# 5) Check size and first few rows
print("rules shape:", rules.shape)
rules.head()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15204\3248675834.py:7: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("cleaned_purchase_data.csv")
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_15204\3248675834.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)
C:\Users\Lenovo\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


rules shape: (1552, 14)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(JAM MAKING SET PRINTED),(6 RIBBONS RUSTIC CHARM),0.055220,0.046611,0.011529,0.208781,4.479269,1.0,0.008955,1.204963,0.822148,0.127671,0.170099,0.228064
1,(6 RIBBONS RUSTIC CHARM),(JAM MAKING SET PRINTED),0.046611,0.055220,0.011529,0.247346,4.479269,1.0,0.008955,1.255265,0.814724,0.127671,0.203355,0.228064
2,(JAM MAKING SET WITH JARS),(6 RIBBONS RUSTIC CHARM),0.053884,0.046611,0.010094,0.187328,4.018997,1.0,0.007582,1.173154,0.793964,0.111658,0.147597,0.201944
3,(6 RIBBONS RUSTIC CHARM),(JAM MAKING SET WITH JARS),0.046611,0.053884,0.010094,0.216561,4.018997,1.0,0.007582,1.207644,0.787906,0.111658,0.171941,0.201944
4,(JUMBO BAG RED RETROSPOT),(6 RIBBONS RUSTIC CHARM),0.102128,0.046611,0.010688,0.104651,2.245223,1.0,0.005928,1.064825,0.617694,0.077419,0.060878,0.166975


In [20]:
rules.to_csv("association_rules_week3.csv",index=False)